In [1]:
import numpy as np
import pandas as pd
import phi

Consider a differential equation of the form

$\partial_t u = \mathcal{L}u + \mathcal{G}(u)$

Formally, we can write the solution to this equation (over one time-step) as

$u(t_n+h) = e^{h\mathcal{L}} u_n + \int_0^h \textrm{d}\tau e^{-\mathcal{L}(\tau-h)} \mathcal{G}(t_n +\tau, u(t_n + \tau))$

where the linear term is treated exactly.

Now the whole problem is how to treat the integral of the non-linear terms.
Using higher order approximations will give better results. A family of Runge-Kutta type integrators can be defined leading to the so-called ETDRK solutions. They are given in terms of the following family of $\varphi_l$ functions, defined as

$\begin{align}
\varphi_0(z) &= e^{z} \\
\varphi_1(z) &= z^{-1}(e^{z} - 1) \\
\varphi_2(z) &= z^{-2}(e^{z} - 1 - z) \\
\varphi_3(z) &= z^{-3}(e^{z} - 1 - z - z^2/2) \\
\cdots &\\
\varphi_l(z) &= \frac{\varphi_{l-1}(z) - \varphi_{l-1}(0)}{z} \\
\varphi_{l+1}(z) &= \frac{\varphi_{l}(z) - 1/k!}{z}
\end{align}$

Unfortunately, these functions suffer from catastrophic error cancellations at the poles. They must be evaluated with extreme care.
The easiest is to use a Talyor expansion around $z=0$ for small values of $z$, and to evaluate the function with increased precision (i.e., long double). Our phi.py package uses the `mpmath` python package to directly evaluate with very high precision, before converting back to double precision.

To first order, we get the following exponential euler integrator 

$\begin{align}
u_{n+1} &= \varphi_0 u_n + h \varphi_1(h\mathcal{L}) \mathcal{G}(t_n, u_n)\\
&\simeq \varphi_0\left(u_n + h \mathcal{G}(t_n,u_n)\right)
\end{align}$


To second order, we have (in Buchner tableau form)
\begin{align}
  \begin{array}{c|cc}
    0 & & \\
    \frac{1}{2} & \frac{1}{2}\varphi_{1,2} & \\
    \hline
      & \varphi_{1} - 2\varphi_{2} & 2\varphi_2
  \end{array}
\end{align}

\begin{align*}
  U_1 &= u_n  &G_{1} &= \mathcal{G}(t_n, U_{1})\\
  U_2 &= \varphi_{0}\left(h/2\mathcal{L}\right) u_n + h\left[\frac{1}{2}\varphi_1\left(h/2 \mathcal{L}\right) G_1\right] & G_2 &= \mathcal{G}(t_n + h/2, U_2) \\
  u_{n+1} &= \varphi_{0} u_n + h\left[\left(\varphi_1 - 2\varphi_2\right)G_1 + 2\varphi_2G_2\right]
\end{align*}

Let us also consider Hochbruck and Ostermann's fourth order method

$\begin{array}{l|llllll}
c_1 = 0 & \\
c_2 = \frac{1}{2} &\frac{1}{2}\varphi_{1,2} \\
c_3 = \frac{1}{2} &\frac{1}{2}\varphi_{1,3}-\varphi_{2,3} & \varphi_{2,3}\\
c_4 = 1 & \varphi_{1,4}-2\varphi_{2,4} & \varphi_{2,4} & \varphi_{2,4} \\
c_5 = \frac{1}{2} & \frac{1}{2}\varphi_{1,5} - \frac{1}{4}\varphi_{2,5}-a_{5,2} & a_{5,2} & a_{5,2} & \frac{1}{4}\varphi_{2,5} - a_{5,2} & \\
\hline
& \varphi_1 - 3\varphi_2 + 4\varphi_3 & 0 & 0 & -\varphi_2 + 4\varphi_3 &4\varphi_2 - 8\varphi_3
\end{array}
$

where $a_{5,2} = \frac{1}{2}\varphi_{2,5} - \varphi_{3,4} + \frac{1}{4}\varphi_{2,4} - \frac{1}{2}\varphi_{3,5}$, and $\varphi_{i,j} = \varphi_i\left(c_j h \mathcal{L}\right)$

The corresponding solution is then

$\begin{align}
U_1 &= u_n & G_1&=\mathcal{G}(t_n, U_1) \\
U_2 &= \varphi_0(h/2 \mathcal{L}) u_n  + h\left[\frac{1}{2} \phi_1(h/2\mathcal{L}) G_1\right] & G_2 &= \mathcal{G}(t_n+h/2, U_2) \\
U_3 &= \varphi_0(h/2\mathcal{L})u_n + h\left[\left(\frac{1}{2}\varphi_1(h/2\mathcal{L}) - \varphi_{2}(h/2\mathcal{L})\right) G_1 + \varphi_2(h/2\mathcal{L}) G_2\right] & G_3 &= \mathcal{G}(t_n+h/2, U_3) \\
U_4 &= \varphi_0 u_n + h\big[\left(\varphi_1 - 2\varphi_2\right) G_1 + \varphi_2 G_2 + \varphi_2 G_3\big] & G_4 &= \mathcal{G}(t_n + h,U_4) \\
U_5 &= \varphi_0(h/2\mathcal{L})u_n + h\left[\left(\frac{1}{2}\varphi_1(h/2\mathcal{L}) - \frac{1}{4}\varphi_2(h/2\mathcal{L}) - a_{5,2}\right)G_1
 + a_{5,2} G_2 + a_{5,2} G_3 + \left(\frac{1}{4} \varphi_2(h/2\mathcal{L}) - a_{5,2}\right) G_4\right] & G_5 &=\mathcal{G}(t_n + h/2, U_5) \\
u_{n+1} &= \varphi_0 u_n + h\big[\left(\varphi_1 - 3\varphi_2 + 4\varphi_3\right) G_1 + 
\left(-\varphi_2 + 4\varphi_3\right)G_4 + (4\varphi_2 - 8\varphi_3)G_5\big]
\end{align}$



References:

[22] M. Hochbruck and A. Ostermann, “Explicit Exponential Runge–Kutta Methods for Semilinear Parabolic
Problems,” SIAM Journal on Numerical Analysis, 2005.

[23] S. M. Cox and P. C. Matthews, “Exponential time differencing for stiff systems,” Journal of Computational Physics, vol. 176, no. 2, pp. 430–455, 2002.

[24] K. Rothauge, E. Haber, and U. Ascher, “The Discrete Adjoint Method for Exponential Integration,” arXiv.org, Oct. 2016.

[25] S. Krogstad, “Generalized integrating factor methods for stiff PDEs,” Journal of Computational Physics, vol. 203, no. 1, pp. 72–88, 2005.

In [2]:
def bad_phi(n, x):
    if n == 0:
        return np.exp(x)
    elif n == 1:
        return (np.exp(x) - 1.0)/x
    elif n == 2:
        return (np.exp(x) - 1.0 - x)/x**2
    elif n == 3:
        return (np.exp(x) - 1.0 - x - x**2/2.0)/x**3
    else:
        print('0<= n < 4')
        return

In [3]:
def load_data():
    data = pd.read_csv('./etdphi4.txt', delim_whitespace=True, dtype=np.float128, comment='#') # computed using mathematica
    sel  = np.abs(data['x']) > .5e-16
    gold = pd.DataFrame()
    for key in data.keys():
        gold[key] = data[key][sel].astype(np.float64)
    gold.reindex()
    return gold
gold = load_data()
gold.style.format("{:18.15e}")
keys = gold.keys()[1:]

In [4]:
bad = pd.DataFrame()
good= pd.DataFrame()
bad['x'] = gold['x']
good['x']= gold['x']
for i,key in enumerate(keys):
    print(i,key)
    bad[key] = bad_phi(i, gold['x'].values)
    good[key]= phi.phin(i, gold['x'].values, dps=100)

0 phi_0
1 phi_1
2 phi_2
3 phi_3


In [5]:
good.style.format("{:18.15e}")

,x,phi_0,phi_1,phi_2,phi_3
0,-1.000000000000000e+00,3.678794411714423e-01,6.321205588285577e-01,3.678794411714423e-01,1.321205588285577e-01
1,-1.000000000000000e-01,9.048374180359595e-01,9.516258196404043e-01,4.837418035959573e-01,1.625819640404268e-01
2,-1.000000000000000e-02,9.900498337491681e-01,9.950166250831947e-01,4.983374916805358e-01,1.662508319464261e-01
3,-1.000000000000000e-03,9.990004998333750e-01,9.995001666250083e-01,4.998333749916681e-01,1.666250083319446e-01
4,-1.000000000000000e-04,9.999000049998333e-01,9.999500016666250e-01,4.999833337499917e-01,1.666625000833319e-01
5,-1.000000000000000e-05,9.999900000499998e-01,9.999950000166666e-01,4.999983333375000e-01,1.666662500008333e-01
6,-1.000000000000000e-06,9.999990000005000e-01,9.999995000001667e-01,4.999998333333750e-01,1.666666250000083e-01
7,-1.000000000000000e-07,9.999999000000050e-01,9.999999500000016e-01,4.999999833333337e-01,1.666666625000001e-01
8,-1.000000000000000e-08,9.999999900000001e-01,9.999999950000000e-01,4.999999983333334e-01,1.666666662500000e-01
9,-1.000000000000000e-09,9.999999990000000e-01,9.999999995000000e-01,4.999999998333333e-01,1.666666666250000e-01


In [6]:
bad_delta = np.abs(gold - bad); bad_delta['x'] = gold['x']
print(bad_delta[keys].max())
bad_delta.style.format("{:.16e}")

phi_0    5.551115e-16
phi_1    1.000000e+00
phi_2    1.000000e+16
phi_3    1.000000e+32
dtype: float64


,x,phi_0,phi_1,phi_2,phi_3
0,-1.0000000000000000e+00,5.5511151231257827e-17,0.0000000000000000e+00,5.5511151231257827e-17,2.7755575615628914e-17
1,-1.0000000000000001e-01,0.0000000000000000e+00,6.6613381477509392e-16,4.8294701571194310e-15,5.0903725679063427e-14
2,-1.0000000000000000e-02,0.0000000000000000e+00,5.3290705182007514e-15,5.3329562987869394e-13,5.3309856529182298e-11
3,-1.0000000000000000e-03,1.1102230246251565e-16,2.9976021664879227e-14,3.0260238759183267e-11,3.0260283973015945e-08
4,-1.0000000000000000e-04,5.5511151231257827e-16,1.3544720900426910e-14,1.3450623947974805e-10,1.3450632853906352e-06
5,-1.0000000000000001e-05,2.2204460492503131e-16,5.7298610300904329e-13,5.7302561307093214e-08,5.7302561265537844e-03
6,-9.9999999999999995e-07,2.2204460492503131e-16,1.5860979196702374e-11,1.5861248027548847e-05,1.5861248027416446e+01
7,-9.9999999999999995e-08,0.0000000000000000e+00,4.8639325900268204e-10,4.8639357734314714e-03,4.8639357734315621e+04
8,-1.0000000000000000e-08,2.2204460492503131e-16,1.0774710013095046e-09,1.0774710085113764e-01,1.0774710085113766e+07
9,-1.0000000000000001e-09,2.2204460492503131e-16,2.7781931755122002e-08,2.7781931525873539e+01,2.7781931525873531e+10


In [7]:
good_delta = np.abs(gold - good); good_delta['x']  = gold['x']
print(good_delta[keys].max())
good_delta.style.format("{:.16e}")

phi_0    5.551115e-16
phi_1    6.661338e-16
phi_2    3.330669e-16
phi_3    1.110223e-16
dtype: float64


,x,phi_0,phi_1,phi_2,phi_3
0,-1.0000000000000000e+00,5.5511151231257827e-17,0.0000000000000000e+00,5.5511151231257827e-17,2.7755575615628914e-17
1,-1.0000000000000001e-01,0.0000000000000000e+00,1.1102230246251565e-16,2.7755575615628914e-16,8.3266726846886741e-17
2,-1.0000000000000000e-02,0.0000000000000000e+00,0.0000000000000000e+00,2.2204460492503131e-16,2.7755575615628914e-17
3,-1.0000000000000000e-03,1.1102230246251565e-16,2.2204460492503131e-16,0.0000000000000000e+00,0.0000000000000000e+00
4,-1.0000000000000000e-04,5.5511151231257827e-16,0.0000000000000000e+00,5.5511151231257827e-17,2.7755575615628914e-17
5,-1.0000000000000001e-05,2.2204460492503131e-16,0.0000000000000000e+00,5.5511151231257827e-17,2.7755575615628914e-17
6,-9.9999999999999995e-07,2.2204460492503131e-16,3.3306690738754696e-16,1.6653345369377348e-16,2.7755575615628914e-17
7,-9.9999999999999995e-08,0.0000000000000000e+00,3.3306690738754696e-16,1.1102230246251565e-16,1.1102230246251565e-16
8,-1.0000000000000000e-08,2.2204460492503131e-16,0.0000000000000000e+00,0.0000000000000000e+00,2.7755575615628914e-17
9,-1.0000000000000001e-09,2.2204460492503131e-16,2.2204460492503131e-16,1.1102230246251565e-16,2.7755575615628914e-17
